In [288]:
import requests
import ast
import pandas as pd
from rich import print as printr
import json
from pathlib import Path

In [289]:
# Define query for Fnugg API.
endpoint = 'https://api.fnugg.no/search'
parameters = {
    'size': 150,
    'sourceFields': 'name,contact,resort_open,region,location,weather_zones,conditions,last_updated',
}

In [290]:
# Access Fnug API and confirm successful GET request.
r = requests.get(endpoint, params=parameters)
print(r.status_code)
data_dict = r.json()

200


In [291]:
print ('Number of resorts matching query = ' + str(len(data_dict['hits']['hits'])))
resorts = data_dict['hits']['hits']

Number of resorts matching query = 121


In [292]:
# Save to file for futre use.
with open('fnugg-data.json', 'w') as json_file:
    json.dump(data_dict, json_file, indent=4)

In [293]:
# Reopen from file.
with open('fnugg-data.json', 'r') as json_file:
    data_dict = json.load(json_file)

In [294]:
# Define DataFrame ro capture key details from API reponse.
columns = ['name', 'region', 'resort_open', 'latitude', 'longitude', 'weather_zone_type', 'snow_at_top_current', 'snow_at_bottom_current', 'last_updated']
df = pd.DataFrame(columns=columns)

# Loop through resorts and collect data.
for resort in resorts:
    row = {
        'name': resort['_source']['name'],
        'region': resort['_source']['region'],
        'resort_open': resort['_source']['resort_open'],
        'latitude': resort['_source']['location']['lat'],
        'longitude': resort['_source']['location']['lon'],
        'weather_zone_type': resort['_source']['weather_zones']['type'],
        'snow_at_top_current': resort['_source']['conditions']['current_report']['top']['snow']['depth_slope'],
        'last_updated': resort['_source']['last_updated'],
    }
    if 'bottom' in resort['_source']['conditions']['current_report']: # Some resorts only have 'top' measurements.
        row['snow_at_bottom_current'] = resort['_source']['conditions']['current_report']['bottom']['snow']['depth_slope']
    else:
        row['snow_at_bottom_current'] = None

    df.loc[len(df)] = row


In [295]:
# Change 'Region' column datatype and clean contents.
# df['region'] = df['region'].astype(str)
# df['region'] = (df['region'].str.strip()
#                 .str.replace("['","")
#                 .str.replace("']","")
#                 .str.replace("[]","")
# )

In [296]:
# Create a DataFrame for the Shiny dashboard.
renamer = {
    'name': 'Name',
    'region': 'Region',
    'resort_open': 'Resort Open',
    'latitude': 'Latitude',
    'longitude': 'Longitude',
    'weather_zone_type': 'Weather Zone Type',
    'snow_at_top_current': 'Snow on Top (cm)',
    'snow_at_bottom_current': 'Snow at Bottom (cm)',
    'last_updated': 'Last Updated',
    }

df_dashboard = df.rename(columns=renamer)
df_dashboard.head()

,Name,Region,Resort Open,Latitude,Longitude,Weather Zone Type,Snow on Top (cm),Snow at Bottom (cm),Last Updated
0,Hallingskarvet Skisenter (Sudndalen),[Østlandet],True,60.652728,8.025943,combined,50,None,2024-01-30T07:39:13Z
1,Fonna Glacier Ski Resort,[],True,60.288106,6.316235,combined,50,None,2024-01-30T07:39:14Z
2,Gausta Skisenter,[Østlandet],True,59.880152,8.734026,combined,100,None,2024-01-30T07:39:13Z
3,Fidjeland Skitrekk,[Sørlandet],True,58.958828,6.945956,combined,70,None,2024-01-30T07:39:14Z
4,Fulufjellet Alpinsenter,[Østlandet],True,61.390801,12.746394,combined,40,None,2024-01-30T07:39:14Z


In [297]:
cwd = Path.cwd()
file_path = cwd / 'dashboard' / 'ski-resorts.csv'

In [298]:
df_dashboard.to_csv(file_path, index=False)

In [299]:
resort_df = pd.read_csv(file_path)

In [300]:
resort_df.dtypes

Name                    object
Region                  object
Resort Open               bool
Latitude               float64
Longitude              float64
Weather Zone Type       object
Snow on Top (cm)       float64
Snow at Bottom (cm)    float64
Last Updated            object
dtype: object

In [301]:
region_selection = ['Midt-Norge', 'Nord-Vestlandet']

In [302]:
def filter_regions(regions, selections):
    regions_list = ast.literal_eval(regions) # regions are initally in string format.
    intersection = set(regions_list) & set(selections)
    return len(intersection) > 0

In [305]:
resort_df['Region'].apply(filter_regions, args=(region_selection,))

0      False
1      False
2      False
3      False
4      False
       ...  
116    False
117    False
118    False
119    False
120    False
Name: Region, Length: 121, dtype: bool

In [310]:
regions = df_dashboard['Region'].explode().unique().tolist()
regions

['Østlandet',
 nan,
 'Sørlandet',
 'Nord-Norge',
 'Nord-Vestlandet',
 'Midt-Norge',
 'Sør-Vestlandet']

In [317]:
regions.remove(None)

ValueError: list.remove(x): x not in list

In [314]:
regions

['Østlandet',
 nan,
 'Sørlandet',
 'Nord-Norge',
 'Nord-Vestlandet',
 'Midt-Norge',
 'Sør-Vestlandet']

In [318]:
del regions[1]

In [319]:
regions

['Østlandet',
 'Sørlandet',
 'Nord-Norge',
 'Nord-Vestlandet',
 'Midt-Norge',
 'Sør-Vestlandet']